In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from pandas import (
    read_sql_query, 
    DataFrame,
    ExcelWriter,
    options
)
from dataframe_image import export


from enum import Enum
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display_html


In [2]:
class Period(Enum):
    start = '2022-01-01'
    end='2022-03-31'
    date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [3]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")



query = f'''
SELECT 
    dh.*,
    dhs.data_name,
    MAX(dhs.date) AS entry_date,
    GROUP_CONCAT(DISTINCT dhs.id_group, '') AS id_group,
    COUNT(*) AS quantity
FROM
    caris_db.dream_hivinfos dh
        LEFT JOIN
    dream_hivinfos_session dhs ON dhs.id = dh.id_dream_hivinfos_session
GROUP BY dhs.data_name, dh.id_patient
HAVING (entry_date >= '{Period.start.value}'
    AND entry_date <= '{Period.end.value}')
'''




dhi1 = read_sql_query(query, engine, parse_dates=True)


# close the pool of connection
engine.dispose()

True

In [4]:
dhi1.id_patient.count()
dhi =  dhi1.groupby(['data_name']).count()
dhi.quantity.sum()

1030

1030

In [5]:
dhi_report = dhi[['quantity']]
dhi_report.reset_index(inplace=True)
dhi_report

,data_name,quantity
0,Accy Nadege,18
1,Benjamin Ketlande,15
2,Charles Joseline Alexis Fresgarline,5
3,Cine Rosemane,14
4,Claude Germane,27
5,Dangervil Charma,17
6,Danise Rene,15
7,Derise Marie Weslaine,49
8,Diego Armando Dorlean,332
9,Dieujuste Wilmène,12


In [6]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")



query = f'''
SELECT 
    dh.*,
    dhs.data_name,
    MAX(dhs.date) AS entry_date,
    GROUP_CONCAT(DISTINCT dhs.id_group, '') AS id_group,
    COUNT(*) AS quantity
FROM
    caris_db.dream_hivinfos dh
        LEFT JOIN
    dream_hivinfos_session dhs ON dhs.id = dh.id_dream_hivinfos_session
GROUP BY dhs.data_name
HAVING (entry_date >= '{Period.start.value}'
    AND entry_date <= '{Period.end.value}')
'''




dhi2 = read_sql_query(query, engine, parse_dates=True)


# close the pool of connection
engine.dispose()

True

In [7]:
Period.end.value

'2022-03-31'

In [8]:
dh_report = dhi2[['data_name','entry_date','quantity']]
dh_report.rename(columns={'quantity':'nombre_infos_perSession'},inplace=True)
dh_report

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_8697/680292962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dh_report.rename(columns={'quantity':'nombre_infos_perSession'},inplace=True)


,data_name,entry_date,nombre_infos_perSession
0,Accy nadege,2022-03-08,223
1,Benjamin Ketlande,2022-03-03,15
2,Charles Joseline Alexis Fresgarline,2022-01-23,5
3,Cine Rosemane,2022-02-12,14
4,Claude Germane,2022-01-15,27
5,Dangervil Charma,2022-02-12,136
6,Danise Rene,2022-02-12,123
7,Derise Marie Weslaine,2022-01-24,4605
8,Diego Armando Dorlean,2022-02-11,3857
9,Dieujuste Wilmène,2022-01-13,25


In [9]:
dh_report.data_name = dh_report.data_name.str.lower()
dhi_report.data_name = dhi_report.data_name.str.lower()



df1_styler = dh_report.style.set_table_attributes("style='display:inline'").set_caption('dh report')
df2_styler = dhi_report.style.set_table_attributes("style='display:inline'").set_caption('dhi report')

display_html(f"{df1_styler._repr_html_()}  LA COMPARAISON VISUELLE  {df2_styler._repr_html_()}", raw=True)


/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_8697/1114690279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dh_report.data_name = dh_report.data_name.str.lower()
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_8697/1114690279.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dhi_report.data_name = dhi_report.data_name.str.lower()


,data_name,entry_date,nombre_infos_perSession
0,accy nadege,2022-03-08,223
1,benjamin ketlande,2022-03-03,15
2,charles joseline alexis fresgarline,2022-01-23,5
3,cine rosemane,2022-02-12,14
4,claude germane,2022-01-15,27
5,dangervil charma,2022-02-12,136
6,danise rene,2022-02-12,123
7,derise marie weslaine,2022-01-24,4605
8,diego armando dorlean,2022-02-11,3857
9,dieujuste wilmène,2022-01-13,25


In [10]:
report_dhi = dhi_report.merge(dh_report, on='data_name',how='left')
report_dhi = report_dhi[['data_name', 'quantity','nombre_infos_perSession','entry_date']]
report_dhi.rename(columns={
        'quantity':'nombre_agyw_perSession' 
    },
        inplace = True
)
report_dhi

,data_name,nombre_agyw_perSession,nombre_infos_perSession,entry_date
0,accy nadege,18,223,2022-03-08
1,benjamin ketlande,15,15,2022-03-03
2,charles joseline alexis fresgarline,5,5,2022-01-23
3,cine rosemane,14,14,2022-02-12
4,claude germane,27,27,2022-01-15
5,dangervil charma,17,136,2022-02-12
6,danise rene,15,123,2022-02-12
7,derise marie weslaine,49,4605,2022-01-24
8,diego armando dorlean,332,3857,2022-02-11
9,dieujuste wilmène,12,25,2022-01-13


In [11]:
report_dhi.to_excel('rapport_dhi.xlsx',index=False,na_rep="")